# **DATATHON 2023: NTT-DATA CHALLENGE**

## **Requirements**:

In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import seaborn as sns

## **Main Program**

Read dataset

In [ ]:
df = pd.read_excel('../assets/consumo_material_clean.xlsx')
df.head()

Some preprocessing

In [ ]:
# Drop rows with NaN values
df = df.dropna()

# Split "ORIGEN" into "REGION", "HOSPITAL" and "DEPARTAMENTO"
df['ORIGEN'] = df['ORIGEN'].str.replace('--', '-')
df[['REGION', 'HOSPITAL', 'DEPARTAMENTO']] = df['ORIGEN'].str.split('-', expand=True)
df = df.drop(["ORIGEN"], axis=1)

# Categorical variables
categorical = ['CODIGO', 'PRODUCTO', 'NUMERO', 'REFERENCIA', 'TIPOCOMPRA', 'REGION', 'HOSPITAL', 'DEPARTAMENTO', 'TGL']
df[categorical] = df[categorical].astype('category')

# Numeric variables
numerical_int = ['CANTIDADCOMPRA', 'UNIDADESCONSUMOCONTENIDAS']
numerical_float = ['PRECIO', 'IMPORTELINEA']
df[numerical_float] = df[numerical_float].astype('float')

# Date format
df['FECHAPEDIDO'] = pd.to_datetime(df['FECHAPEDIDO'], format='%d/%m/%y')
df['MES'] = df['FECHAPEDIDO'].dt.month
df['AÑO'] = df['FECHAPEDIDO'].dt.year
df = df.drop('FECHAPEDIDO', axis=1)

# Create new variable
df['PRECIOUNIDAD'] = df['IMPORTELINEA'] / df['CANTIDADCOMPRA']

df.to_csv('../assets/preprocessed_df.csv', index=False)

df.head()

New dataset with groupby

In [ ]:
new_df = df[['PRODUCTO', 'FECHAPEDIDO', 'TIPOCOMPRA', 'REGION', 'HOSPITAL', 'DEPARTAMENTO', 'TGL', 'CANTIDADCOMPRA', 'UNIDADESCONSUMOCONTENIDAS', 'PRECIO', 'IMPORTELINEA']].copy()

new_df = new_df.groupby(['AÑO', 'MES', 'PRODUCTO', 'HOSPITAL', 'TIPOCOMPRA', 'TGL'], observed=True).agg({'CANTIDADCOMPRA': 'sum', 'UNIDADESCONSUMOCONTENIDAS': 'mean', 'PRECIO': 'mean', 'IMPORTELINEA': 'sum', 'PRECIOUNIDAD': 'mean'}).reset_index()

new_df.drop(['UNIDADESCONSUMOCONTENIDAS', 'PRECIO', 'IMPORTELINEA', 'PRECIOUNIDAD'], axis=1, inplace=True)

new_df.to_csv('../assets/new_df.csv', index=False)

new_df.head()


Split train and test datasets

In [ ]:
split_year = 2023
train = new_df.loc[new_df['AÑO'] < split_year]
test = new_df.loc[new_df['AÑO'] >= split_year]

In [ ]:
train.to_csv('../assets/train.csv', index=False)
test.to_csv('../assets/test.csv', index=False)